<a href="https://colab.research.google.com/github/JackWittmayer/PrincipleLearning/blob/main/principle_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Part 0: Set up and Hello World

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from typing import List, Optional

In [ ]:
DRIVE_FOLDER = "/content/drive/MyDrive/colab/"
MODEL_NAME = "Qwen/Qwen3-4B-Instruct-2507"
MODEL_PATH = DRIVE_FOLDER + MODEL_NAME
TOKENIZER_PATH = DRIVE_FOLDER + MODEL_NAME + "-Tokenizer"

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
def load_model_and_tokenizer(download_fresh: bool):
    if download_fresh:
        model = AutoModelForCausalLM.from_pretrained(
            MODEL_NAME,
            dtype="auto",
            device_map="auto"
        )
        tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
        model.save_pretrained(MODEL_PATH)
        tokenizer.save_pretrained(TOKENIZER_PATH)
    else:
        model = AutoModelForCausalLM.from_pretrained(
            MODEL_PATH,
            dtype="auto",
            device_map="auto"
        )
        tokenizer = AutoTokenizer.from_pretrained(TOKENIZER_PATH)
    return model, tokenizer
model, tokenizer = load_model_and_tokenizer(download_fresh=False)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
def generate_response(user_prompt: str, system_prompt: Optional[str] = None) -> str:
    if system_prompt:
        messages = [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ]
    else:
        messages = [
            {"role": "user", "content": user_prompt}
        ]
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True,
    )
    model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

    # conduct text completion
    generated_ids = model.generate(
        **model_inputs,
        max_new_tokens=16384
    )
    output_ids = generated_ids[0][len(model_inputs.input_ids[0]):].tolist()

    # conduct text completion
    generated_ids = model.generate(
        **model_inputs,
        max_new_tokens=16384
    )
    output_ids = generated_ids[0][len(model_inputs.input_ids[0]):].tolist()
    content = tokenizer.decode(output_ids, skip_special_tokens=True)
    return content

In [ ]:
prompt = "Give me a short introduction to large language models."
content = generate_response(prompt)
print("content:", content)

content: Large language models (LLMs) are artificial intelligence systems trained on vast amounts of text data from the internet, books, articles, and other sources. These models learn to understand and generate human-like language by identifying patterns in how words and sentences are used. Using deep learning techniques—particularly transformer architectures—LLMs can perform a wide range of tasks, such as answering questions, writing stories, coding, summarizing text, and translating languages. Their ability to generate coherent and contextually relevant responses makes them powerful tools in natural language processing. Examples include models like GPT-3, BERT, and Llama. While highly capable, they still require careful use to avoid inaccuracies, biases, or misinformation.


# Part 1: Adding a principle into the system prompt.

In [ ]:
def create_principle_prompt(principles: List[str]) -> str:
    prompt = """
    You are the personal AI assistant to a person who uses the following
    personal principles to guide their decision making.
    [
    """
    for n, principle in enumerate(principles):
        prompt += f"{n}. {principle}\n"
    prompt += """
    ]
    Carefully follow these principles when completing tasks and generating responses
    as if you were this person. Don't explicitly reference the principles.
    """
    return prompt


In [ ]:
minimalist_principle = "Acquire minimal material goods."
collector_principle = "Embrace abundance without guilt."
question_one = "Should I keep a box of books I never read?"

In [ ]:
minimalist_response = generate_response(
    user_prompt = question_one,
    system_prompt = create_principle_prompt([minimalist_principle])
)
collector_response = generate_response(
    user_prompt = question_one,
    system_prompt = create_principle_prompt([collector_principle])
)
print("minimalist response:", minimalist_response)
print("collector response:", collector_response)

# Part 2: LoRA Instruction Fine-Tuning

## Creating the dataset

In [ ]:
!pip install replicate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.6/48.6 kB 3.0 MB/s eta 0:00:00


In [ ]:
from google.colab import userdata
import os
replicate_api_token = userdata.get("REPLICATE_API_TOKEN")
if not replicate_api_token:
    raise ValueError("REPLICATE_API_TOKEN not found in Colab secrets.")
os.environ["REPLICATE_API_TOKEN"] = replicate_api_token

In [ ]:
import random
random.seed(25)
def generate_prompts(num_prompts: int = 500) -> List[str]:
    # Categories of prompts where principles matter
    prompt_templates = {
        "shopping_decisions": [
            "I'm thinking about buying {item}. Should I get it?",
            "There's a sale on {item}. Is it worth buying?",
            "I saw a beautiful {item} at the store. What do you think?",
            "My {item} broke. Should I replace it with something nicer?",
            "I found a vintage {item} online. Should I purchase it?",
        ],

        "gifts_and_receiving": [
            "Someone gave me {item} as a gift, but I don't really need it. What should I do?",
            "My family wants to know what I want for my birthday. Any ideas?",
            "A friend is offering me their old {item}. Should I take it?",
            "I received duplicate {item} as gifts. What should I do with the extras?",
        ],

        "home_and_space": [
            "My apartment feels {feeling}. Any suggestions?",
            "I'm moving to a new place. How should I approach setting it up?",
            "I have some empty {space} in my home. What should I do with it?",
            "Should I keep or get rid of {item} taking up space?",
            "I'm redecorating my {room}. Where should I start?",
        ],

        "decluttering": [
            "I'm doing a big declutter. How should I decide what to keep?",
            "I haven't used {item} in months. Should I keep it?",
            "My closet is getting full. What's your advice?",
            "I'm feeling overwhelmed by all my stuff. Help?",
            "Should I keep things that have sentimental value but no practical use?",
        ],

        "financial_decisions": [
            "I just got a bonus at work. What should I do with the money?",
            "I'm trying to save money. Where should I cut back?",
            "Is it worth spending more on a higher quality {item}?",
        ],

        "hobbies_and_collections": [
            "I'm interested in starting {hobby}. What equipment do I need?",
            "Should I expand my collection of {collectible}?",
            "I found a rare {collectible}. Should I buy it?",
            "How many {item} is too many?",
            "Is it okay to have multiple hobbies that require equipment?",
        ],

        "lifestyle_changes": [
            "I'm moving to a smaller place. How should I prepare?",
            "I'm moving to a bigger place. How should I use the extra space?",
            "I want to change my lifestyle. Where should I start?",
            "How can I make my space feel more like 'me'?",
        ],

        "social_pressure": [
            "My friends all have {trendy_item}. Should I get one too?",
            "Everyone at work has {item}. Am I missing out?",
            "My partner wants to buy {item} but I'm not sure. What should I say?",
            "My family thinks I need more {category} in my life. Are they right?",
        ],

        "sustainability_vs_enjoyment": [
            "Is it wasteful to buy {item} if I already have something similar?",
            "Should I prioritize experiences or possessions?",
            "How do I balance enjoying life with not being materialistic?",
            "I feel guilty about wanting {item}. Should I?",
        ],

        "quality_of_life": [
            "Would my life be better with {upgrade}?",
            "I'm considering upgrading my {item}. Is it worth it?",
            "What items actually improve quality of life?",
            "Should I invest in making my space more comfortable?",
        ]
    }

    # Specific items and variables to fill templates
    items = [
        "a new couch", "artwork", "decorative pillows", "a coffee table book",
        "kitchen gadgets", "another pair of shoes", "a bookshelf", "plants",
        "candles", "throw blankets", "wall art", "a standing desk",
        "noise-canceling headphones", "a smartwatch", "designer clothing",
        "collectible figures", "board games", "vinyl records", "cameras",
        "sporting equipment", "craft supplies", "kitchen appliances",
        "furniture", "tech accessories", "jewelry", "books", "video games"
    ]

    feelings = ["empty", "cluttered", "boring", "cramped", "uninspiring"]
    spaces = ["shelves", "walls", "corners", "countertops"]
    rooms = ["living room", "bedroom", "home office", "kitchen"]
    hobbies = ["photography", "painting", "woodworking", "gaming", "cooking", "reading"]
    collectibles = ["books", "vinyl records", "art prints", "figurines", "vintage items"]
    trendy_items = ["an air fryer", "a robot vacuum", "smart home devices", "standing desks"]
    categories = ["furniture", "decorations", "hobbies", "stuff"]
    upgrades = ["a better mattress", "nicer dishes", "upgraded furniture", "better lighting"]

    # Generate prompts
    prompts = []

    while len(prompts) < num_prompts:
        for category, templates in prompt_templates.items():
            for template in templates:
                # Generate multiple variations of each template
                for _ in range(max(1, num_prompts // (len(prompt_templates) * len(templates)))):
                    prompt = template.format(
                        item=random.choice(items),
                        feeling=random.choice(feelings),
                        space=random.choice(spaces),
                        room=random.choice(rooms),
                        hobby=random.choice(hobbies),
                        collectible=random.choice(collectibles),
                        trendy_item=random.choice(trendy_items),
                        category=random.choice(categories),
                        upgrade=random.choice(upgrades)
                    )
                    prompts.append(prompt)
        prompts = list(set(prompts))

    # Shuffle and return requested number
    random.shuffle(prompts)
    return prompts[:num_prompts] if num_prompts < len(prompts) else prompts

prompts = generate_prompts(100)
print(len(prompts))
for prompt in prompts[:10]:
    print(prompt)

100
Should I keep or get rid of noise-canceling headphones taking up space?
I'm considering upgrading my a smartwatch. Is it worth it?
A friend is offering me their old candles. Should I take it?
I found a rare books. Should I buy it?
I'm moving to a bigger place. How should I use the extra space?
I received duplicate a new couch as gifts. What should I do with the extras?
There's a sale on vinyl records. Is it worth buying?
I'm feeling overwhelmed by all my stuff. Help?
How many sporting equipment is too many?
Would my life be better with a better mattress?


In [ ]:
import replicate
import json
from typing import List, Dict

def generate_preference_pair(prompt: str, principle: str, anti_principle: str) -> Dict:
    """Generate a preference pair for DPO training using Replicate"""
    model_name = "anthropic/claude-4.5-haiku"

    # Generate chosen response (aligned with principle)
    chosen_response = replicate.run(
        model_name,
        input={
            "prompt": prompt,
            "system_prompt": create_principle_prompt([principle]),
            "max_new_tokens": 1024,
        },
        api_token=replicate_api_token
    )
    chosen_response_text = "".join(chosen_response)

    # Generate rejected response (aligned with opposite principle)
    rejected_response = replicate.run(
        model_name,
        input={
            "prompt": prompt,
            "system_prompt": create_principle_prompt([anti_principle]),
            "max_new_tokens": 1024,
        },
        api_token=replicate_api_token
    )
    rejected_response_text = "".join(rejected_response)


    return {
        "prompt": prompt,
        "chosen": chosen_response_text,
        "rejected": rejected_response_text
    }

# Generate dataset
dataset = []
for n, prompt in enumerate(prompts):
    pair = generate_preference_pair(
        prompt,
        principle=minimalist_principle,
        anti_principle=collector_principle
    )
    dataset.append(pair)
    print(f"Created pair {n}")
    print("PROMPT:")
    print(pair["prompt"])
    print()
    print("CHOSEN:")
    print(pair["chosen"])
    print()
    print("REJECTED:")
    print(pair["rejected"])
    print()
    # Write to file
    with open(DRIVE_FOLDER + "dataset.json", "w") as f:
        json.dump(dataset, f)

Created pair 0
PROMPT:
Should I keep or get rid of noise-canceling headphones taking up space?

CHOSEN:
I'd lean toward getting rid of them if they're just sitting there taking up space.

Ask yourself honestly:
- **Do you actually use them regularly?** If they're gathering dust, they're not earning their physical footprint in your life.
- **Could you borrow or rent them when needed?** Noise-canceling headphones are common enough that options exist if you occasionally need them.
- **What would you gain by removing them?** Freed space, reduced mental clutter, possibly some money if you sell them.

The key question isn't "might I need this someday?" but "am I genuinely using this now?" Keeping things "just in case" tends to be what fills spaces without adding real value to daily life.

If you do use them regularly—commuting, flights, focused work—then they're earning their keep and worth the space. But if they're more of a "nice to have" that sits idle, that's a clean candidate for lettin

In [ ]:
from IPython.display import display, Markdown
display(Markdown("# Prompt 0"))
display(Markdown(dataset[0]["prompt"]))
display(Markdown("# Chosen"))
display(Markdown(dataset[0]["chosen"]))
display(Markdown("# Rejected"))
display(Markdown(dataset[0]["rejected"]))

# Prompt 0

Should I keep or get rid of noise-canceling headphones taking up space?

# Chosen

I'd lean toward getting rid of them if they're just sitting there taking up space.

Ask yourself honestly:
- **Do you actually use them regularly?** If they're gathering dust, they're not earning their physical footprint in your life.
- **Could you borrow or rent them when needed?** Noise-canceling headphones are common enough that options exist if you occasionally need them.
- **What would you gain by removing them?** Freed space, reduced mental clutter, possibly some money if you sell them.

The key question isn't "might I need this someday?" but "am I genuinely using this now?" Keeping things "just in case" tends to be what fills spaces without adding real value to daily life.

If you do use them regularly—commuting, flights, focused work—then they're earning their keep and worth the space. But if they're more of a "nice to have" that sits idle, that's a clean candidate for letting go.

# Rejected

I'd think about this practically: How often do you actually use them? 

If they're genuinely useful—reducing distractions during work, making travel better, improving your audio experience—they're earning their space. The value they provide outweighs the physical space they occupy.

If you find yourself reaching for them regularly or missing them when you don't have them, that's a sign to keep them.

If they've been sitting unused for months, they're probably just clutter at that point, and getting rid of them frees up space for something you'll actually use.

The key question: does having and using them improve your life? If yes, keep them without second-guessing the space. If no, let them go.